In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from torchvision import models
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!cd ../input/berlin-database-of-emotional-speech-emodb

In [4]:

data_dir = "../input/berlin-database-of-emotional-speech-emodb/wav"
file_path_list = os.listdir(data_dir)
print(len(np.unique(file_path_list)))
file_path_list = [data_dir + "/" + file_path for file_path in file_path_list]
EMOTIONS = {1:'angry', 2:'bored', 3:'disgust', 4:'fear', 5:'happy', 6:'sad', 0:'neutral'} 
# DATA_PATH = '../input/ravdess-emotional-speech-audio/'
SAMPLE_RATE = 48000

# data = pd.DataFrame(columns=['letter', 'motion', 'Gender','Path'])
# for filename in file_path_list:
print(len(file_path_list))

In [5]:
emotion = []
path = []
for root, dirs, files in os.walk("../input/berlin-database-of-emotional-speech-emodb/wav"):
    print(len(root), len(dirs), len(files))
    for name in files:
#         print(name)
#         if name[0:2] in '0310111215':  # MALE
#             if name[5] == 'W':  # Ärger (Wut) -> Angry
#                 emotion.append('angry')
#             elif name[5] == 'L':  # Langeweile -> Boredom
#                 emotion.append('bored')
#             elif name[5] == 'E':  # Ekel -> Disgusted
#                 emotion.append('disgust')
#             elif name[5] == 'A':  # Angst -> Angry
#                 emotion.append('fear')
#             elif name[5] == 'F':  # Freude -> Happiness
#                 emotion.append('happy')
#             elif name[5] == 'T':  # Trauer -> Sadness
#                 emotion.append('sad')
#             elif name[6] == 'N':
#                 emotion.append('neutral')
#             else:
#                 emotion.append('unknown')
#         else:
        if name[5] == 'W':  # Ärger (Wut) -> Angry
            emotion.append('angry')
        elif name[5] == 'L':  # Langeweile -> Boredom
            emotion.append('bored')
        elif name[5] == 'E':  # Ekel -> Disgusted
            emotion.append('disgust')
        elif name[5] == 'A':  # Angst -> Angry
            emotion.append('fear')
        elif name[5] == 'F':  # Freude -> Happiness
            emotion.append('happy')
        elif name[5] == 'T':  # Trauer -> Sadness
            emotion.append('sad')
        elif name[5] == 'N':
            emotion.append('neutral')
        else:
            emotion.append('unknown')

        path.append(os.path.join("../input/berlin-database-of-emotional-speech-emodb/wav", name))

emodb_df = pd.DataFrame(emotion, columns=['Emotion'])
emodb_df['source'] = 'EMODB'
# emodb_df = pd.concat([emodb_df, pd.DataFrame(path, columns=['Path'])], axis=1)
emodb_df["Path"] = path

In [6]:
emodb_df
data = emodb_df.dropna()
data

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(x=range(7), height=emodb_df['Emotion'].value_counts())
ax.set_xticks(ticks=range(7))
ax.set_xticklabels([EMOTIONS[i] for i in range(7)],fontsize=10)
ax.set_xlabel('Emotions')
ax.set_ylabel('Number of examples')

In [8]:
data = emodb_df
#Dropping rows in df with NaN values
data = data.dropna()
print(len(data))

In [9]:
print(data.head())

In [10]:
data['Emotion'].value_counts()

In [11]:

mel_spectrograms = []
signals = []
for i, file_path in enumerate(data.Path):
    audio, sample_rate = librosa.load(file_path, duration=3, offset=0.5, sr=SAMPLE_RATE)
    signal = np.zeros((int(SAMPLE_RATE*3,)))
    signal[:len(audio)] = audio
    signals.append(signal)
    print("\r Processed {}/{} files".format(i,len(data)),end='')
signals = np.stack(signals,axis=0)
print(type(signals))
print(signals)

In [12]:
X = signals
X = np.array(X)
train_ind,test_ind,val_ind = [],[],[]
X_train,X_val,X_test = [],[],[]
Y_train,Y_val,Y_test = [],[],[]
for emotion in range(len(EMOTIONS)):
    em = EMOTIONS[emotion]
#     print(em)
    emotion_ind = list(data.loc[data.Emotion==em,'Emotion'].index)
#     print(np.sum(data.Emotion==em))
#     print(emotion_ind)
    emotion_ind = np.random.permutation(emotion_ind)
    m = len(emotion_ind)
    ind_train = emotion_ind[:int(0.8*m)]
#     print(ind_train)
    ind_val = emotion_ind[int(0.8*m):int(0.9*m)]
    ind_test = emotion_ind[int(0.9*m):]
    X_train.append(X[ind_train,:])
    Y_train.append(np.array([emotion]*len(ind_train),dtype=np.int32))
    X_val.append(X[ind_val,:])
    Y_val.append(np.array([emotion]*len(ind_val),dtype=np.int32))
    X_test.append(X[ind_test,:])
    Y_test.append(np.array([emotion]*len(ind_test),dtype=np.int32))
    train_ind.append(ind_train)
    test_ind.append(ind_test)
    val_ind.append(ind_val)
X_train = np.concatenate(X_train,0)
X_val = np.concatenate(X_val,0)
X_test = np.concatenate(X_test,0)
Y_train = np.concatenate(Y_train,0)
Y_val = np.concatenate(Y_val,0)
Y_test = np.concatenate(Y_test,0)
train_ind = np.concatenate(train_ind,0)
val_ind = np.concatenate(val_ind,0)
test_ind = np.concatenate(test_ind,0)
print(f'X_train:{X_train.shape}, Y_train:{Y_train.shape}')
print(f'X_val:{X_val.shape}, Y_val:{Y_val.shape}')
print(f'X_test:{X_test.shape}, Y_test:{Y_test.shape}')
# check if all are unique
unique, count = np.unique(np.concatenate([train_ind,test_ind,val_ind],0), return_counts=True)
print("Number of unique indexes is {}, out of {}".format(sum(count==1), X.shape[0]))

del X

In [13]:
def addAWGN(signal, num_bits=16, augmented_num=2, snr_low=15, snr_high=30): 
    signal_len = len(signal)
    # Generate White Gaussian noise
    noise = np.random.normal(size=(augmented_num, signal_len))
    # Normalize signal and noise
    norm_constant = 2.0**(num_bits-1)
    signal_norm = signal / norm_constant
    noise_norm = noise / norm_constant
    # Compute signal and noise power
    s_power = np.sum(signal_norm ** 2) / signal_len
    n_power = np.sum(noise_norm ** 2, axis=1) / signal_len
    # Random SNR: Uniform [15, 30] in dB
    target_snr = np.random.randint(snr_low, snr_high)
    # Compute K (covariance matrix) for each noise 
    K = np.sqrt((s_power / n_power) * 10 ** (- target_snr / 10))
    K = np.ones((signal_len, augmented_num)) * K  
    # Generate noisy signal
    return signal + K.T * noise

In [14]:
aug_signals = []
aug_labels = []
for i in range(X_train.shape[0]):
    signal = X_train[i,:]
    augmented_signals = addAWGN(signal)
    for j in range(augmented_signals.shape[0]):
        em = data.loc[i, "Emotion"]
        aug_labels.append(list(EMOTIONS.keys())[list(EMOTIONS.values()).index(em)])
        aug_signals.append(augmented_signals[j,:])
        data = data.append(data.iloc[i], ignore_index=True)
    print("\r Processed {}/{} files".format(i,X_train.shape[0]),end='')
aug_signals = np.stack(aug_signals,axis=0)
X_train = np.concatenate([X_train,aug_signals],axis=0)
aug_labels = np.stack(aug_labels,axis=0)
Y_train = np.concatenate([Y_train,aug_labels])
print('')
print(f'X_train:{X_train.shape}, Y_train:{Y_train.shape}')

In [15]:
X_train = X_train[:(32 * (X_train.shape[0]//32))]
Y_train = Y_train[:(32 * (Y_train.shape[0]//32))]
X_val = X_val[:(32 * (X_val.shape[0]//32))]
Y_val = Y_val[:(32 * (Y_val.shape[0]//32))]
X_test = X_test[:(32 * (X_test.shape[0]//32))]
Y_test = Y_test[:(32 * (Y_test.shape[0]//32))]

In [16]:

def getMELspectrogram(audio, sample_rate):
    mel_spec = librosa.feature.melspectrogram(y=audio,
                                              sr=sample_rate,
                                              n_fft=1024,
                                              win_length = 512,
                                              window='hamming',
                                              hop_length = 256,
                                              n_mels=128,
                                              fmax=sample_rate/2
                                             )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# test function
audio, sample_rate = librosa.load(data.loc[0,'Path'], duration=3, offset=0.5,sr=SAMPLE_RATE)
signal = np.zeros((int(SAMPLE_RATE*3,)))
signal[:len(audio)] = audio
mel_spectrogram = getMELspectrogram(signal, SAMPLE_RATE)
librosa.display.specshow(mel_spectrogram, y_axis='mel', x_axis='time')
print('MEL spectrogram shape: ',mel_spectrogram.shape)

In [17]:
mel_train = []
print("Calculatin mel spectrograms for train set")
for i in range(X_train.shape[0]):
    mel_spectrogram = getMELspectrogram(X_train[i,:], sample_rate=SAMPLE_RATE)
    mel_train.append(mel_spectrogram)
    print("\r Processed {}/{} files".format(i,X_train.shape[0]),end='')
print('')
del X_train

mel_val = []
print("Calculatin mel spectrograms for val set")
for i in range(X_val.shape[0]):
    mel_spectrogram = getMELspectrogram(X_val[i,:], sample_rate=SAMPLE_RATE)
    mel_val.append(mel_spectrogram)
    print("\r Processed {}/{} files".format(i,X_val.shape[0]),end='')
print('')
del X_val

mel_test = []
print("Calculatin mel spectrograms for test set")
for i in range(X_test.shape[0]):
    mel_spectrogram = getMELspectrogram(X_test[i,:], sample_rate=SAMPLE_RATE)
    mel_test.append(mel_spectrogram)
    print("\r Processed {}/{} files".format(i,X_test.shape[0]),end='')
print('')
del X_test

In [18]:
def splitIntoChunks(mel_spec,win_size,stride):
    t = mel_spec.shape[1]
    num_of_chunks = int(t/stride)
    chunks = []
    for i in range(num_of_chunks):
        chunk = mel_spec[:,i*stride:i*stride+win_size]
        if chunk.shape[1] == win_size:
            chunks.append(chunk)
    return np.stack(chunks,axis=0)

In [19]:
# get chunks
# train set
mel_train_chunked = []
for mel_spec in mel_train:
    chunks = splitIntoChunks(mel_spec, win_size=128,stride=64)
    mel_train_chunked.append(chunks)
print("Number of chunks is {}".format(chunks.shape[0]))
# val set
mel_val_chunked = []
for mel_spec in mel_val:
    chunks = splitIntoChunks(mel_spec, win_size=128,stride=64)
    mel_val_chunked.append(chunks)
print("Number of chunks is {}".format(chunks.shape[0]))
# test set
mel_test_chunked = []
for mel_spec in mel_test:
    chunks = splitIntoChunks(mel_spec, win_size=128,stride=64)
    mel_test_chunked.append(chunks)
print("Number of chunks is {}".format(chunks.shape[0]))

In [20]:
import torch
import torch.nn as nn
# BATCH FIRST TimeDistributed layer
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)
        # squash samples and timesteps into a single axis
        elif len(x.size()) == 3: # (samples, timesteps, inp1)
            x_reshape = x.contiguous().view(-1, x.size(2))  # (samples * timesteps, inp1)
        elif len(x.size()) == 4: # (samples,timesteps,inp1,inp2)
            x_reshape = x.contiguous().view(-1, x.size(2), x.size(3)) # (samples*timesteps,inp1,inp2)
        else: # (samples,timesteps,inp1,inp2,inp3)
            x_reshape = x.contiguous().view(-1, x.size(2), x.size(3),x.size(4)) # (samples*timesteps,inp1,inp2,inp3)
            
        y = self.module(x_reshape)
        
        # we have to reshape Y
        if len(x.size()) == 3:
            y = y.contiguous().view(x.size(0), -1, y.size(1))  # (samples, timesteps, out1)
        elif len(x.size()) == 4:
            y = y.contiguous().view(x.size(0), -1, y.size(1), y.size(2)) # (samples, timesteps, out1,out2)
        else:
            y = y.contiguous().view(x.size(0), -1, y.size(1), y.size(2),y.size(3)) # (samples, timesteps, out1,out2, out3)
        return y

In [21]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [22]:
pretrained_alexnet = models.alexnet(pretrained = True)
pretrained_alexnet.eval

In [23]:
set_parameter_requires_grad(pretrained_alexnet, True)
num_ftrs2 = pretrained_alexnet.classifier[6].in_features
pretrained_alexnet.classifier[6] = nn.Linear(num_ftrs2, 1024)
pretrained_alexnet.eval

In [24]:
pretrained = models.resnet18(pretrained = True)
set_parameter_requires_grad(pretrained, True)
num_ftrs = pretrained.fc.in_features
pretrained.fc = nn.Linear(num_ftrs, 1024)

In [25]:
class HybridModel(nn.Module):
    def __init__(self,num_emotions):
        super().__init__()
        # conv block
        #self.alexnet = pretrained
        self.alexnet = pretrained_alexnet
        self.conv_2d_1 = nn.Sequential(
            nn.Conv2d(in_channels = 1,
                      out_channels = 3,
                      kernel_size = 3,
                      stride = 1,
                      padding = 1)
        )
        self.conv2Dblock1 = nn.Sequential(
            # 1. conv block
            TimeDistributed(nn.Conv2d(in_channels=1,
                                   out_channels=16,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  )),
            TimeDistributed(nn.BatchNorm2d(16)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(kernel_size=2, stride=2)),
            TimeDistributed(nn.Dropout(p=0.3)))
        self.conv2Dblock2 = nn.Sequential(
            TimeDistributed(nn.Conv2d(in_channels=16,
                                   out_channels=32,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  )),
            TimeDistributed(nn.BatchNorm2d(32)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(kernel_size=4, stride=4)),
            TimeDistributed(nn.Dropout(p=0.3)),
        )
        self.conv2Dblock3 = nn.Sequential(
            TimeDistributed(nn.Conv2d(in_channels=32,
                                   out_channels=64,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  )),
            TimeDistributed(nn.BatchNorm2d(64)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(kernel_size=4, stride=4)),
            TimeDistributed(nn.Dropout(p=0.3))
        )
        # LSTM block
        hidden_size = 64
        self.lstm = nn.LSTM(input_size=1024,hidden_size=hidden_size,bidirectional=True, batch_first=True)
        self.dropout_lstm = nn.Dropout(p=0.4)
        self.attention_linear = nn.Linear(2*hidden_size,1) # 2*hidden_size for the 2 outputs of bidir LSTM
        # Linear softmax layer
        self.out_linear = nn.Linear(2*hidden_size,num_emotions)

    def forward(self,x):

        # print("shape x:", x.shape)
        td_op = TimeDistributed(self.conv_2d_1)(x)
        # print("Time distributed op: ", td_op.shape)

        td_op = torch.reshape(td_op, [224, 3, 128, 128])
        # print("Time distributed op: ", td_op.shape)
        # conv_op = self.conv_2d_1(x)
        # print("Convolution output: ", td_op)
        # print(td_op.shape)
        # print(type(td_op))
        alexnet_out = self.alexnet(td_op)
        # print("shape after td resnet:", alexnet_out.shape)
        #alexnet_out = alexnet_out.reshape()
        alexnet_output = torch.reshape(alexnet_out, [32, 7, 1024])
        # print("After reshape: ", alexnet_output.shape)
        # conv_embedding = self.conv2Dblock1(x)
        # print("shape after b1:", conv_embedding.shape)
        # conv_embedding = self.conv2Dblock2(conv_embedding)
        # print("shape after b2:", conv_embedding.shape)
        # conv_embedding = self.conv2Dblock3(conv_embedding)
        # print("shape after b3:", conv_embedding.shape)
        # conv_embedding = torch.flatten(conv_embedding, start_dim=2) # do not flatten batch dimension and time
        # print("Conv embedding: ", conv_embedding.shape)
        # lstm_embedding, (h,c) = self.lstm(conv_embedding)
        lstm_embedding, (h,c) = self.lstm(alexnet_output)
        lstm_embedding = self.dropout_lstm(lstm_embedding)
        # lstm_embedding (batch, time, hidden_size*2)
        # print("LSTM Embedding: ", lstm_embedding.shape)
        batch_size,T,_ = lstm_embedding.shape 
        attention_weights = [None]*T
        for t in range(T):
            embedding = lstm_embedding[:,t,:]
            attention_weights[t] = self.attention_linear(embedding)
        attention_weights_norm = nn.functional.softmax(torch.stack(attention_weights,-1),dim=-1)
        attention = torch.bmm(attention_weights_norm, lstm_embedding) # (Bx1xT)*(B,T,hidden_size*2)=(B,1,2*hidden_size)
        attention = torch.squeeze(attention, 1)
        output_logits = self.out_linear(attention)
        output_softmax = nn.functional.softmax(output_logits,dim=1)
        # print(output_logits.shape, output_softmax.shape, attention_weights_norm.shape)
        return output_logits, output_softmax, attention_weights_norm

In [26]:
def loss_fnc(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions,target=targets)
def make_train_step(model, loss_fnc, optimizer):
    def train_step(X,Y):
        # set model to train mode
        model.train()
        # forward pass
        output_logits, output_softmax, attention_weights_norm = model(X)
        predictions = torch.argmax(output_softmax,dim=1)
        accuracy = torch.sum(Y==predictions)/float(len(Y))
        # compute loss
        loss = loss_fnc(output_logits, Y)
        # compute gradients
        loss.backward()
        # update parameters and zero gradients
        optimizer.step()
        optimizer.zero_grad()
        return loss.item(), accuracy*100
    return train_step

In [27]:
def make_validate_fnc(model,loss_fnc):
    def validate(X,Y):
      EPOCHS=50
      DATASET_SIZE = X_val.shape[0]
      BATCH_SIZE = 32
      device = 'cuda' if torch.cuda.is_available() else 'cpu'
      losses = []
      preds = []
      accs = []
      iters = int(DATASET_SIZE / BATCH_SIZE)
      for i in range(iters):
          batch_start = i * BATCH_SIZE
          batch_end = min(batch_start + BATCH_SIZE, DATASET_SIZE)
          actual_batch_size = batch_end-batch_start
          x = X[batch_start:batch_end,:,:,:,:]
          y = Y[batch_start:batch_end]
          X_tensor = torch.tensor(x,device=device).float()
          Y_tensor = torch.tensor(y, dtype=torch.long,device=device)
          # loss, acc = train_step(X_tensor,Y_tensor)
          # epoch_acc += acc*actual_batch_size/DATASET_SIZE
          # epoch_loss += loss*actual_batch_size/DATASET_SIZE
          # print(f"\r Epoch {epoch}: iteration {i}/{iters}",end='')
    
          with torch.no_grad():
              model.eval()
              print("Valid X shape: ", x.shape)
              output_logits, output_softmax, attention_weights_norm = model(x)
              predictions = torch.argmax(output_softmax,dim=1)
              accuracy = torch.sum(y==predictions)/float(len(y))
              loss = loss_fnc(output_logits,y)
              print(f"Batch num: {i}\nAccuracy: {accuracy}")
              losses.append(loss.item())
              accs.append(accuracy.to('cpu'))
              preds.extend(predictions.to('cpu'))
      acc = torch.mean(torch.Tensor(accs))
      loss = torch.sum(torch.Tensor(losses))
      preds = torch.Tensor(preds)
      return loss, acc*100, preds
    return validate

In [28]:
X_train = np.stack(mel_train_chunked,axis=0)
X_train = np.expand_dims(X_train,2)
print('Shape of X_train: ',X_train.shape)
X_val = np.stack(mel_val_chunked,axis=0)
X_val = np.expand_dims(X_val,2)
print('Shape of X_val: ',X_val.shape)
X_test = np.stack(mel_test_chunked,axis=0)
X_test = np.expand_dims(X_test,2)
print('Shape of X_test: ',X_test.shape)

del mel_train_chunked
del mel_train
del mel_val_chunked
del mel_val
del mel_test_chunked
del mel_test

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

b,t,c,h,w = X_train.shape
X_train = np.reshape(X_train, newshape=(b,-1))
X_train = scaler.fit_transform(X_train)
X_train = np.reshape(X_train, newshape=(b,t,c,h,w))

b,t,c,h,w = X_test.shape
X_test = np.reshape(X_test, newshape=(b,-1))
X_test = scaler.transform(X_test)
X_test = np.reshape(X_test, newshape=(b,t,c,h,w))

b,t,c,h,w = X_val.shape
X_val = np.reshape(X_val, newshape=(b,-1))
X_val = scaler.transform(X_val)
X_val = np.reshape(X_val, newshape=(b,t,c,h,w))

In [30]:
print(X_train.shape)
type(X_train)

In [31]:
EPOCHS= 125
DATASET_SIZE = X_train.shape[0]
BATCH_SIZE = 32
print('Selected device is {}'.format(device))
model = HybridModel(num_emotions=len(EMOTIONS)).to(device)
print('Number of trainable params: ',sum(p.numel() for p in model.parameters()))
#OPTIMIZER = torch.optim.Adam(model.parameters(),lr=1e-5, weight_decay=1e-4)

OPTIMIZER = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay=1e-3, momentum=0.8)

train_step = make_train_step(model, loss_fnc, optimizer=OPTIMIZER)
validate = make_validate_fnc(model,loss_fnc)
losses=[]
val_losses = []
accs = []
val_accs = []
for epoch in range(EPOCHS):
    # schuffle data
    ind = np.random.permutation(DATASET_SIZE)
    X_train = X_train[ind,:,:,:,:]
    Y_train = Y_train[ind]
    epoch_acc = 0
    epoch_loss = 0
    iters = int(DATASET_SIZE / BATCH_SIZE)
    for i in range(iters):
        batch_start = i * BATCH_SIZE
        batch_end = min(batch_start + BATCH_SIZE, DATASET_SIZE)
        actual_batch_size = batch_end-batch_start
#         print(batch_start)
#         print(batch_end)
        X = X_train[batch_start:batch_end,:,:,:,:]
        Y = Y_train[batch_start:batch_end]
#         print(Y)
#         print(Y.shape)
#         print(type(Y))
        X_tensor = torch.tensor(X,device=device).float()
        Y_tensor = torch.tensor(Y, dtype=torch.long,device=device)
        loss, acc = train_step(X_tensor, Y_tensor)
        epoch_acc += acc*actual_batch_size/DATASET_SIZE
        epoch_loss += loss*actual_batch_size/DATASET_SIZE
        print(f"\r Epoch {epoch}: iteration {i}/{iters}",end='')
    X_val_tensor = torch.tensor(X_val,device=device).float()
    Y_val_tensor = torch.tensor(Y_val,dtype=torch.long,device=device)
    val_loss, val_acc, _ = validate(X_val_tensor,Y_val_tensor)
    accs.append(epoch_acc)
    losses.append(epoch_loss)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print('')
    print(f"Epoch {epoch} --> loss:{epoch_loss:.4f}, acc:{epoch_acc:.2f}%, val_loss:{val_loss:.4f}, val_acc:{val_acc:.2f}%")

In [32]:
# Model Statistics
plt.plot(val_losses, label = 'valid_loss')
plt.plot(losses, label = 'train_loss')
plt.legend()
plt.show()

In [33]:
# Model Statistics
plt.plot(val_accs, label = 'valid_accuracy')
plt.plot(list(map(lambda x: x.cpu(), accs)), label = 'train_accuracy')
plt.legend()
plt.show()